# 1. Data loading

In [9]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

question_df = pd.read_csv("data/raw/questions.csv")

# 2. LLM challenger Answer a question

In [3]:
import uuid
from src.consts import SELECTED_LLMS
from src.answer import llms_answer_questions, answer_questions, extract_answer

/home/lambertbruyas/git_perso/gpt-pursuit/data-acquisition/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BATCH_SIZE = 1
question_sample = question_df.set_index('id').sample(BATCH_SIZE, random_state=42)

In [5]:
SELECTED_LLMS[4:]

[{'pipeline_kwargs': {'task': 'text-generation',
   'model': 'google/gemma-2-2b-it',
   'tokenizer': 'google/gemma-2-2b-it'},
  'model_size': 2000000000},
 {'pipeline_kwargs': {'task': 'text-generation',
   'model': 'Qwen/Qwen2.5-1.5B-Instruct',
   'tokenizer': 'Qwen/Qwen2.5-1.5B-Instruct'},
  'model_size': 1540000000}]

In [6]:
for llm_config in SELECTED_LLMS[4:]:
    model_name = llm_config["pipeline_kwargs"]["model"]
    question_answers = answer_questions(question_sample["Question"].tolist(), llm_config)
    llm_answer_df = pd.DataFrame(question_answers)
    llm_answer_df.columns = [cname+"_"+model_name for cname in llm_answer_df.columns]
    llm_answer_df["question_id"] = question_sample.reset_index()["id"]
    llm_answer_df.to_csv(f"data/processed/{uuid.uuid4()}.csv", index=False)

llm_answer_df

/home/lambertbruyas/git_perso/gpt-pursuit/data-acquisition/models/google/gemma-2-2b-it
Can't load locally pipeline for google/gemma-2-2b-it: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/home/lambertbruyas/git_perso/gpt-pursuit/data-acquisition/models/google/gemma-2-2b-it'. Use `repo_type` argument if needed.. Downloading it.


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.78s/it]
Device set to use cpu


Saved pipeline for google/gemma-2-2b-it
/home/lambertbruyas/git_perso/gpt-pursuit/data-acquisition/models/Qwen/Qwen2.5-1.5B-Instruct
Can't load locally pipeline for Qwen/Qwen2.5-1.5B-Instruct: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/home/lambertbruyas/git_perso/gpt-pursuit/data-acquisition/models/Qwen/Qwen2.5-1.5B-Instruct'. Use `repo_type` argument if needed.. Downloading it.


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cpu


Saved pipeline for Qwen/Qwen2.5-1.5B-Instruct


,answer_Qwen/Qwen2.5-1.5B-Instruct,time_taken_Qwen/Qwen2.5-1.5B-Instruct,question_id
0,"[{'role': 'user', 'content': 'You are particip...",0 days 00:00:07.434117,4414
